
The Proteus Toolkit![triton](http://proteustoolkit.org/_static/corps_logo_cross2.svg)
=================

This notebook is also a (live) slide presentation. To view it as a presentation, click on the button in the toolbar that looks like this: ![rise button](rise.png)

Use the space bar to step through and shift-enter to execute code cells.

What is  Proteus?
=================

A Python toolkit for solving equations

Find $u:\Omega \rightarrow \mathbb{R}$ satisfying
$$
\nabla \cdot (-a\nabla u) + r(x) = 0
$$
on 
$$
\Omega = [0,1] \times [0,1]
$$
subject to
$$
u|_{\partial \Omega} = 0
$$

First, import some tools. 

Remember to click on the cell and hit shift-enter to execute code.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from proteus.iproteus import *
import proteus.default_p as physics 
import proteus.default_n as numerics 
from proteus.TransportCoefficients import PoissonEquationCoefficients

Define the equation

In [ ]:
def r(x):
    return (-32.*np.pi*2*
            np.sin(np.pi*2*x[...,0])*
            np.sin(np.pi*2*x[...,1])*
            np.cos(np.pi*2*x[...,0])*
            np.cos(np.pi*2*x[...,1]))
def a(x):
    return np.array([[1.0,0.0],
                     [0.0,1.0]])
physics.coefficients = PoissonEquationCoefficients([a],[r]) 
physics.name="PoissonDots"

Define the domain and boundary conditions

In [ ]:
physics.nd = 2
physics.domain =  Domain.RectangularDomain(L=[1.0,1.0],
                                           x=[0.0,0.0])
def getDBC(x,flag):
    if flag in [physics.domain.boundaryTags['left'],
                physics.domain.boundaryTags['top'],
                physics.domain.boundaryTags['right'],
                physics.domain.boundaryTags['bottom']]:
        return lambda x,t: 0.0
        
physics.dirichletConditions = {0:getDBC}

Configure the numerics

In [ ]:
numerics=default_n
numerics.femSpaces = {0:FemTools.P1}
numerics.elementQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd,3)
numerics.elementBoundaryQuadrature = Quadrature.SimplexGaussQuadrature(physics.nd-1,3)
numerics.nn = 41
numerics.numericalFluxType = NumericalFlux.Diffusion_SIPG_exterior
so = default_so
so.name = physics.name
so.sList=[default_s]
so.tnList = [0,1]

Solve the problem

In [ ]:
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run1')
assert(not failed)

Plot the solution

In [ ]:
model = ns.modelList[0].levelModelList[-1]
plt.figure()
plt.tricontourf(model.mesh.nodeArray[:,0],
                model.mesh.nodeArray[:,1],
                model.mesh.elementNodesArray,
                model.u[0].dof)

A Python toolkit for research on ***continuum models***. The next cell defines a new continuum model equations (a nonlinear Poisson equation).

In [ ]:
class NonlinearPoisson(TransportCoefficients.TC_base):
    def __init__(self,A,rofx):
        TransportCoefficients.TC_base.__init__(self, 
                         nc=1, 
                         variableNames=['u'],
                         diffusion = {0:{0:{0:'constant'}}},
                         potential = {0:{0:'nonlinear'}},
                         reaction  = {0:{0:'constant'}})
        self.A=A;
        self.rofx=rofx
    def evaluate(self,t,c):
        c[('a',0,0)][...,0] = self.A[0][0]
        c[('a',0,0)][...,3] = self.A[1][1]
        c[('r',0)][:]       = self.rofx(c['x'][:])
        c[('phi',0)][:]     = c[('u',0)]**2+10.0*c[('u',0)]
        c[('dphi',0,0)][:]  = 2.0*c[('u',0)] + 10.0

In [ ]:
physics.coefficients = NonlinearPoisson([[1.0,0.0],[0.0,1.0]], r)
numerics.tolFac=0.0
numerics.nl_atol_res = 1.0e-8
numerics.nn=21

In [ ]:
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run2')
assert(not failed)

In [ ]:
model = ns.modelList[0].levelModelList[-1]
plt.figure()
plt.tricontourf(model.mesh.nodeArray[:,0],
                model.mesh.nodeArray[:,1],
                model.mesh.elementNodesArray,
                model.u[0].dof) 

A Python toolkit for research on ***numerical methods***. The next cell switches the discretation to a new finite element (P2, or continuous quadratics on triangles with the Lagrange basis).

In [ ]:
physics.coefficients = PoissonEquationCoefficients([a],[r]) 
numerics.femSpaces={0:FemTools.P2}
ns = NumericalSolution.NS_base(so,[physics],[numerics],so.sList,opts)
failed = ns.calculateSolution('run3')
assert(not failed)

In [ ]:
model = ns.modelList[0].levelModelList[-1]
import matplotlib.tri as tri
triang = tri.Triangulation(model.u[0].femSpace.dofMap.lagrangeNodesArray[:,0],
                model.u[0].femSpace.dofMap.lagrangeNodesArray[:,1])

plt.figure()
plt.tricontourf(triang, model.u[0].dof)

These examples were intented to be simple and general illustrations of the some of the main goals of the proteus project:

- Enabling R&D on new continuum models (PDEs)
- Enabling R&D on new numerical methods for existing models

This notebook directory contains additional tutorials and examples.

https://github.com/erdc/proteus